# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack_find').getOrCreate()

In [0]:
from pyspark.ml.clustering import KMeans
dataset = spark.read.csv("dbfs:/FileStore/shared_uploads/arunyuvi0206@gmail.com/hack_data.csv",header=True,inferSchema=True)

In [0]:
dataset.head()

Out[6]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [0]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [0]:
dataset.columns

Out[8]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']
             

In [0]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [0]:
final_data = vec_assembler.transform(dataset)

In [0]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [0]:
final_data = vec_assembler.transform(dataset)

In [0]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(final_data)
cluster_final_data = scalerModel.transform(final_data)

In [0]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [0]:
def point_to_centroid_distance(point, centroid):
    return sum((point[i] - centroid[i]) ** 2 for i in range(len(point)))

def cluster_cost(cluster, centroid):
    return sum(point_to_centroid_distance(point, centroid) for point in cluster)

def compute_wssse(model, data):
    centroids = model.clusterCenters()
    clusters = model.transform(data).rdd.map(lambda r: (r.prediction, Vectors.dense(r.features)))
    
    wssse = clusters.map(lambda pair: cluster_cost([pair[1]], centroids[pair[0]])).sum()
    return wssse


In [0]:
wssse_k3 = compute_wssse(model_k3,cluster_final_data)
wssse_k2 = compute_wssse(model_k2,cluster_final_data)
wssse_k2
wssse_k3

Out[31]: 150815444.45329437

In [0]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    wssse = compute_wssse(model ,cluster_final_data)
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)


With K=2
Within Set Sum of Squared Errors = 150815803.25554746
------------------------------------------------------------
With K=3
Within Set Sum of Squared Errors = 150815444.45329437
------------------------------------------------------------
With K=4
Within Set Sum of Squared Errors = 150815301.06827962
------------------------------------------------------------
With K=5
Within Set Sum of Squared Errors = 150806619.41766334
------------------------------------------------------------
With K=6
Within Set Sum of Squared Errors = 150806457.77685565
------------------------------------------------------------
With K=7
Within Set Sum of Squared Errors = 150796670.56776017
------------------------------------------------------------
With K=8
Within Set Sum of Squared Errors = 150790346.77273896
------------------------------------------------------------


In [0]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [0]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()


+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

